In [232]:
import urllib.request
from bs4 import BeautifulSoup
from pandas import DataFrame
import numpy as np
from geopy.geocoders import Nominatim

In [62]:
url = 'https://en.wikipedia.org/wiki/List_of_science_fiction_conventions'

In [63]:
ufile = urllib.request.urlopen(url)

In [64]:
newtext = str(ufile.read())

In [65]:
soup = BeautifulSoup(newtext,'lxml')

In [196]:
data = []
table = soup.find('table', attrs={'class':'wikitable sortable'})
rows = table.find_all('tr')
realrows = rows[1:]
for row in realrows:
    colsraw = row.find_all('td')
    cols = [ele.text.strip() for ele in colsraw]
    if colsraw[4].a:  cols[4] = colsraw[4].a.get('href') 
    data.append(cols)

In [199]:
df = DataFrame(data)

In [200]:
df[-5:]

,0,1,2,3,4,5
131,WisCon,"Madison, Wisconsin",USA,General,http://www.wiscon.info,2015-05-22
132,Wizard World,Various,USA,Media,http://www.wizardworld.com/,Various
133,WonderCon,"Los Angeles, California",USA,SF/F,http://www.comic-con.org/wc,2015-04-03
134,World Fantasy Convention,"Saratoga Springs, New York",USA,Fantasy,http://www.worldfantasy.org/,2015-11-05
135,Worldcon,Various,,SF/General,http://www.worldcon.org/,2015-08-19


In [207]:
domestic = df[(df=='USA').any(1)]

In [230]:
texan = domestic[domestic[1].str.contains('Texas')]

In [231]:
texan

,0,1,2,3,4,5
2,AggieCon,"College Station, Texas",USA,General,http://aggiecon.tamu.edu/,2016-04-01
4,All-Con,"Dallas, Texas",USA,General,http://www.all-con.net,2016-03-17
5,ApolloCon,"Houston, Texas",USA,General,http://www.apollocon.org,2015-06-19
9,ArmadilloCon,"Austin, Texas",USA,SF LIT,http://www.armadillocon.org,2015-07-24
26,ConDFW,"Dallas, Texas",USA,SF LIT,http://www.condfw.org/,2016-02-12
50,FenCon,"Dallas, Texas",USA,General,http://www.fencon.org,2015-09-25
67,Lazy Dragon Con,"McKinney, Texas",USA,General,http://www.lazydragon.com/con/,2008-07-18
70,Linucon,"Austin, Texas",USA,General,,2005-09-30
121,UnCommonCon,"Dallas, Texas",USA,SF/F,,2000-11-24


In [233]:
geolocator = Nominatim()

In [266]:
coords = []
address = []
county = []

In [267]:
for place in texan[1]:
    location = geolocator.geocode(place)
    coords.append([location.latitude,location.longitude])
    address.append(location.address)

In [268]:
coords

[[30.6262616, -96.3347248],
 [32.7762719, -96.7968558],
 [29.7589382, -95.3676973],
 [30.2711286, -97.7436994],
 [32.7762719, -96.7968558],
 [32.7762719, -96.7968558],
 [33.1976496, -96.615447],
 [30.2711286, -97.7436994],
 [32.7762719, -96.7968558]]

In [269]:
for line in address:
    spec = line.split(',')
    newspec = [bit.strip().replace(' County','') for bit in spec]
    county.append(newspec[1])

In [270]:
county

['Brazos',
 'Dallas',
 'Harris',
 'Travis',
 'Dallas',
 'Dallas',
 'Collin',
 'Travis',
 'Dallas']

In [273]:
txdata = DataFrame({'Con': texan[0], 'Location': texan[1],'County': county,'Coordinates': coords})

In [274]:
txdata

,Con,Coordinates,County,Location
2,AggieCon,"[30.6262616, -96.3347248]",Brazos,"College Station, Texas"
4,All-Con,"[32.7762719, -96.7968558]",Dallas,"Dallas, Texas"
5,ApolloCon,"[29.7589382, -95.3676973]",Harris,"Houston, Texas"
9,ArmadilloCon,"[30.2711286, -97.7436994]",Travis,"Austin, Texas"
26,ConDFW,"[32.7762719, -96.7968558]",Dallas,"Dallas, Texas"
50,FenCon,"[32.7762719, -96.7968558]",Dallas,"Dallas, Texas"
67,Lazy Dragon Con,"[33.1976496, -96.615447]",Collin,"McKinney, Texas"
70,Linucon,"[30.2711286, -97.7436994]",Travis,"Austin, Texas"
121,UnCommonCon,"[32.7762719, -96.7968558]",Dallas,"Dallas, Texas"


In [275]:
txdata.to_csv('TexasCons.csv')